In [38]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import tensorflow as tf

In [39]:
# Hyperparameters
split = 0.85
sequence_length = 10
epochs = 10
learning_rate = 0.02

In [40]:
# Loading stock price data
stock_data = pd.read_csv("historical_data.csv")
columns = ['Close']
ticker_column = 'Ticker'

In [41]:
# Encode the tickers
label_encoder = LabelEncoder()
stock_data['Ticker_Encoded'] = label_encoder.fit_transform(stock_data[ticker_column])

In [42]:
# Splitting data into train and test sets
stock_data['Split'] = stock_data.groupby('Ticker').cumcount() / stock_data.groupby('Ticker')['Date'].transform('count')
train_data = stock_data[stock_data['Split'] < split]
test_data = stock_data[stock_data['Split'] >= split]

In [43]:
# Normalize numerical data
scaler = MinMaxScaler()
train_data.loc[:, 'Close'] = scaler.fit_transform(train_data[['Close']])
test_data.loc[:, 'Close'] = scaler.transform(test_data[['Close']])

In [44]:
# Prepare sequences for training and testing
def prepare_sequences(data, sequence_length, scaler):
    X_seq, X_ticker, y = [], [], []
    grouped = data.groupby('Ticker')
    for _, group in grouped:
        group_values = group[columns].values
        ticker_values = group['Ticker_Encoded'].values
        for i in range(len(group_values) - sequence_length):
            X_seq.append(group_values[i:i + sequence_length])
            X_ticker.append(ticker_values[i])
            y.append(group_values[i + sequence_length])
    return np.array(X_seq), np.array(X_ticker), np.array(y)

In [45]:
X_train, X_ticker_train, y_train = prepare_sequences(train_data, sequence_length, scaler)
X_test, X_ticker_test, y_test = prepare_sequences(test_data, sequence_length, scaler)

In [46]:
# Model creation with an embedding layer
def model_create(vocab_size, embedding_dim=8):
    tf.random.set_seed(1234)
    
    # Sequential input for stock data
    seq_input = tf.keras.Input(shape=(X_train.shape[1], 1), name="Sequence_Input")
    
    # Ticker embedding input
    ticker_input = tf.keras.Input(shape=(1,), name="Ticker_Input")
    ticker_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1)(ticker_input)
    ticker_embedding_flat = tf.keras.layers.Flatten()(ticker_embedding)
    
    # LSTM layers
    lstm_out = tf.keras.layers.LSTM(units=50, activation="tanh", return_sequences=True)(seq_input)
    lstm_out = tf.keras.layers.Dropout(0.15)(lstm_out)
    lstm_out = tf.keras.layers.LSTM(units=30, activation="tanh", return_sequences=True)(lstm_out)
    lstm_out = tf.keras.layers.Dropout(0.05)(lstm_out)
    lstm_out = tf.keras.layers.LSTM(units=20, activation="tanh", return_sequences=False)(lstm_out)
    lstm_out = tf.keras.layers.Dropout(0.01)(lstm_out)
    
    # Combine embeddings and LSTM output
    combined = tf.keras.layers.Concatenate()([lstm_out, ticker_embedding_flat])
    
    # Dense layers
    dense_out = tf.keras.layers.Dense(units=50, activation="relu")(combined)
    dense_out = tf.keras.layers.Dropout(0.2)(dense_out)
    output = tf.keras.layers.Dense(units=1, activation="linear")(dense_out)
    
    # Compile the model
    model = tf.keras.models.Model(inputs=[seq_input, ticker_input], outputs=output)
    model.compile(
        loss="mean_squared_error",
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    )
    
    # Train the model
    model.fit(
        [X_train, X_ticker_train], y_train,
        epochs=epochs,
        batch_size=32,
        verbose=1
    )
    return model

In [47]:
# Invert normalization
y_test = scaler.inverse_transform(y_test)

In [48]:
# Prediction on test set
def predict(model):
    predictions = model.predict([X_test, X_ticker_test])
    predictions = scaler.inverse_transform(predictions.reshape(-1, 1)).reshape(-1, 1)
    return predictions

In [49]:
# Evaluation
def evaluate(predictions):
    mae = mean_absolute_error(predictions, y_test)
    mape = mean_absolute_percentage_error(predictions, y_test)
    return mae, mape, (1 - mape)

In [50]:
# Trial runs
def run_model(n):
    total_mae = total_mape = total_acc = 0
    vocab_size = len(label_encoder.classes_)
    for i in range(n):
        model = model_create(vocab_size)
        predictions = predict(model)
        mae, mape, acc = evaluate(predictions)
        total_mae += mae
        total_mape += mape 
        total_acc += acc 
    return (total_mae / n), (total_mape / n), (total_acc / n), predictions.tolist()

In [51]:
mae, mape, acc, preds = run_model(1)

print(f"Mean Absolute Error = {mae}")
print(f"Mean Absolute Percentage Error = {mape*100:.2f}%")
print(f"Accuracy = {acc}")

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - loss: 0.0017
Epoch 2/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 9.5608e-04
Epoch 3/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0012
Epoch 4/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0014
Epoch 5/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 9.4674e-04
Epoch 6/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 9.7362e-04
Epoch 7/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 9.4056e-04
Epoch 8/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 0.0010
Epoch 9/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 8.4903e-04
Epoch 10/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 9.3985e-04
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Mean Absolute Error = 39.493361078289276
Mean Absolute Percentage Error = 32.53%
Accuracy = 0.6747156904704079
